In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL
import pathlib
from PIL import Image
from pathlib import Path
from tensorflow.keras import layers,models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
loaded_data = Path("C:\\Users\\FARAZ\\Desktop\\ml-course\\dl\\data")

In [3]:
total_image_count = 0

# Check if there are any subfolders (flower categories)
flower_categories = list(loaded_data.glob('*'))
if len(flower_categories) > 0:
    for flower_category in flower_categories:
        flower_images = list(flower_category.glob('*.jpg')) + list(flower_category.glob('*.png'))
        if len(flower_images) > 0:
            total_image_count += len(flower_images)
    print(f"Total number of flower images found: {total_image_count}")
else:
    print("No flower categories (subfolders) found in the directory.")

set_height, set_width = 180, 180
batch_size=32

Total number of flower images found: 444


In [4]:
training_images = tf.keras.preprocessing.image_dataset_from_directory(
    loaded_data,
    subset="training",
    validation_split = 0.25,
    seed=123,
    image_size=(set_height, set_width),
    batch_size=batch_size
)


Found 444 files belonging to 6 classes.
Using 333 files for training.


In [5]:
validation_images = tf.keras.preprocessing.image_dataset_from_directory(
    loaded_data,
    subset="validation",  # Use "validation" for the validation subset
    validation_split=0.25,  # Adjust this value as needed
    seed=123,
    image_size=(set_height, set_width),
    batch_size=batch_size
)


Found 444 files belonging to 6 classes.
Using 111 files for validation.


In [6]:
flower_classes = training_images.class_names
print(flower_classes)

['.ipynb_checkpoints', 'daisy', 'dandelions', 'roses', 'sunflowers', 'tulips']


In [7]:
dataset_classes = 6

In [8]:
model = Sequential([])

layers.experimental.preprocessing.Rescaling(1./255, input_shape=(set_height, set_width, 3))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(dataset_classes,activation='softmax'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs=10

CNN_model = model.fit(
  training_images,
  validation_data=validation_images,
  epochs=epochs
)


Epoch 1/10


C:\Users\FARAZ\AppData\Roaming\Python\Python310\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


11/11 [==============================] - 7s 591ms/step - loss: 65.8032 - accuracy: 0.3123 - val_loss: 2.7055 - val_accuracy: 0.3423
Epoch 2/10
11/11 [==============================] - 6s 487ms/step - loss: 1.6914 - accuracy: 0.3874 - val_loss: 1.5907 - val_accuracy: 0.3514
Epoch 3/10
11/11 [==============================] - 5s 462ms/step - loss: 1.3013 - accuracy: 0.5766 - val_loss: 1.4394 - val_accuracy: 0.3514
Epoch 4/10
11/11 [==============================] - 5s 466ms/step - loss: 0.8812 - accuracy: 0.6607 - val_loss: 1.4670 - val_accuracy: 0.3874
Epoch 5/10
11/11 [==============================] - 5s 441ms/step - loss: 0.3998 - accuracy: 0.8589 - val_loss: 1.7643 - val_accuracy: 0.4324
Epoch 6/10
11/11 [==============================] - 5s 452ms/step - loss: 0.1427 - accuracy: 0.9550 - val_loss: 1.6783 - val_accuracy: 0.4595
Epoch 7/10
11/11 [==============================] - 5s 454ms/step - loss: 0.0544 - accuracy: 0.9940 - val_loss: 2.4150 - val_accuracy: 0.4505
Epoch 8/10
11/11

In [9]:
def predict_input_image(img):
  img_4d=img.reshape(-1,180,180,3)
  prediction=model.predict(img_4d)[0]
  return {flower_classes[i]: float(prediction[i]) for i in range(5)}

In [ ]:
import gradio as gr

# Define the input image
image = gr.inputs.Image(shape=(180, 180))

# Define the output label
label = gr.outputs.Label(num_top_classes=5)

# Create the Gradio interface
interface= gr.Interface(fn=predict_input_image, inputs=image, outputs=label, interpretation='default')
interface.launch(debug=True)


C:\Users\FARAZ\AppData\Local\Temp\ipykernel_10984\2342002876.py:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(180, 180))
C:\Users\FARAZ\AppData\Local\Temp\ipykernel_10984\2342002876.py:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(180, 180))
C:\Users\FARAZ\AppData\Local\Temp\ipykernel_10984\2342002876.py:7: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label = gr.outputs.Label(num_top_classes=5)
C:\Users\FARAZ\AppData\Local\Temp\ipykernel_10984\2342002876.py:7: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label = gr.outputs.Label(num_top_classes=5)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 57ms/step
